<a href="https://colab.research.google.com/github/Varun221/LeNet/blob/master/LeNet_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LeNet-5
This is an implementation of the Research paper by Yann LeCunn et al\
------ so and so matter to be added ------

![alt text](https://drive.google.com/uc?export=view&id=16CQ1IskrfIqqN-277w3G5X_l1B8mN-1S)

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

In [ ]:
 # Loading the mnist dataset
 train, test = tfds.as_numpy(tfds.load(
    'mnist',
    split=['train','test'], 
    batch_size=-1, 
    as_supervised=True,
))
trainX, trainY = train[0], train[1]
testX, testY = test[0], test[1]
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.0. Subsequent calls will reuse this data.
(60000, 28, 28, 1) (60000,) (10000, 28, 28, 1) (10000,)


NameError: ignored